# Introduction: Using Trained Model

The purpose of this noteobook is to use the trained word level model in order to make predictions. We can look at using both the model trained with pre-trained embeddings and the model with embeddings that were trained from scratch.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [21]:
from keras import Model
from keras.models import load_model

BATCH_SIZE = 2048
RANDOM_STATE = 50

import numpy as np
import pandas as pd
from utils import get_model, find_closest, get_sequences, create_train_valid

# Model Trained with Own Embeddings

In [5]:
model, embeddings, word_idx, idx_word = get_model('training-rnn')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         1619200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 16192)             2088768   
Total params: 3,841,728
Trainable params: 3,841,728
Non-trainable params: 0
_________________________________________________________________


/home/ubuntu/git/recurrent-neural-networks/notebooks/utils.py:18: RuntimeWarning: invalid value encountered in true_divide
  embeddings = embeddings / np.linalg.norm(embeddings, axis = 1).reshape((-1, 1))


In [6]:
find_closest('the', embeddings, word_idx, idx_word)

Query: the

Word: the             Cosine Similarity: 1.0
Word: a               Cosine Similarity: 0.8669000267982483
Word: this            Cosine Similarity: 0.8206999897956848
Word: entire          Cosine Similarity: 0.7939000129699707
Word: its             Cosine Similarity: 0.791100025177002
Word: no              Cosine Similarity: 0.7544999718666077
Word: second          Cosine Similarity: 0.7511000037193298
Word: new             Cosine Similarity: 0.7408999800682068
Word: another         Cosine Similarity: 0.7401999831199646
Word: The             Cosine Similarity: 0.7389000058174133


In [7]:
find_closest('a', embeddings, word_idx, idx_word)

Query: a

Word: a               Cosine Similarity: 1.0
Word: the             Cosine Similarity: 0.8669000267982483
Word: A               Cosine Similarity: 0.8044000267982483
Word: another         Cosine Similarity: 0.8019999861717224
Word: this            Cosine Similarity: 0.7854999899864197
Word: any             Cosine Similarity: 0.7379999756813049
Word: no              Cosine Similarity: 0.7372000217437744
Word: its             Cosine Similarity: 0.7357000112533569
Word: second          Cosine Similarity: 0.7120000123977661
Word: unprecedented   Cosine Similarity: 0.7078999876976013


In [8]:
find_closest('network', embeddings, word_idx, idx_word)

Query: network

Word: network         Cosine Similarity: 1.0
Word: networks        Cosine Similarity: 0.8083000183105469
Word: channel         Cosine Similarity: 0.8008999824523926
Word: cable           Cosine Similarity: 0.7594000101089478
Word: program         Cosine Similarity: 0.7491999864578247
Word: system          Cosine Similarity: 0.7479000091552734
Word: same            Cosine Similarity: 0.7031999826431274
Word: media           Cosine Similarity: 0.7001000046730042
Word: link            Cosine Similarity: 0.6970999836921692
Word: web             Cosine Similarity: 0.692799985408783


In [9]:
find_closest('neural', embeddings, word_idx, idx_word)

Query: neural

Word: neural          Cosine Similarity: 1.0
Word: lower-layer     Cosine Similarity: 0.6693999767303467
Word: RETE            Cosine Similarity: 0.6685000061988831
Word: TCP/IP          Cosine Similarity: 0.666100025177002
Word: retro-causal    Cosine Similarity: 0.6539000272750854
Word: Neural          Cosine Similarity: 0.647599995136261
Word: fuzzy-neural    Cosine Similarity: 0.6473000049591064
Word: neuronal        Cosine Similarity: 0.6272000074386597
Word: 3G              Cosine Similarity: 0.6029000282287598
Word: self-stabilizing Cosine Similarity: 0.590499997138977


In [10]:
find_closest('UNK', embeddings, word_idx, idx_word)

UNK has no pre-trained embedding.


In [24]:
data = pd.read_csv('../data/neural_network_patent_query.csv')
abstracts = list(data['patent_abstract'])
features, labels, sequences = get_sequences(abstracts, model_name = 'training-rnn')

There are 16192 unique words.
There are 318563 training sequences.


In [25]:
' '.join([idx_word[x] for x in features[0]])

'A Barometer Neuron enhances stability in a Neural Network System that , when used as a track-while-scan system , assigns sensor plots to predicted track positions in a plot/track association situation . The Barometer Neuron functions as a bench-mark or reference system node that equates a superimposed plot and track'

In [26]:
idx_word[np.argmax(labels[0])]

'to'

In [27]:
preds = model.predict(features[0].reshape((1, 50)))
preds.shape

(1, 16192)

In [28]:
idx_word[np.argmax(preds)]

'of'

In [29]:
from utils import generate_output

In [32]:
from IPython.display import HTML

In [33]:
seed_html, gen_html, a_html = generate_output(model, sequences, idx_word)
HTML(seed_html)
HTML(gen_html)
HTML(a_html)

/home/ubuntu/git/recurrent-neural-networks/notebooks/utils.py:220: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / diversity


In [20]:
X_train, X_test, y_train, y_test = create_train_valid(features, labels, num_words = len(word_idx))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
titles[1805]

# Injecting Diversity into Predictions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def diversify(arr, diversity, plot = False):
    div = np.log(arr) / diversity
    exp_preds = np.exp(div)
    preds = exp_preds / np.sum(exp_preds)
    if plot:
        plt.figure(figsize = (10, 8));
        plt.subplot(2, 1, 1);
        sns.distplot(arr); plt.title('Original Distribution');
        plt.subplot(2, 1, 2);
        sns.distplot(preds); plt.title(f'Distribution with {diversity} diversity')
    probas = np.random.multinomial(1, preds, 1);
    return probas

In [ ]:
_ = diversify([0.1, 0.2, 0.4, 0.8, 0.9, 0.1, 0.3, 0.4], 2, plot = True)

In [ ]:
x = diversify(pred[0, 0, :], 10, True)
index_word[np.argmax(x)]

In [ ]:
for d in np.arange(1, 5, 0.2):
    x = diversify(pred[0, 0, :], d)
    print(index_word[np.argmax(x)])

In [ ]:
x = diversify(pred[0, 0, :], 1, True)
index_word[np.argmax(x)]

In [ ]:
np.argmax(x)

In [ ]:
sorted_p = np.sort(pred)

In [ ]:
inputBatch = """[[0. 12184.   420.  4636.     0.  8840.     0.     0. 10499. 11508.
   7511.     0.  5468.  2879.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  6689.  2818. 12003.  6480.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.  3045. 11087.  2710.     0.   494.  1087.   420.  4995.
  11516.  3637.  5842.     0.  9963.  7015. 11090.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  1287.   420.  4070. 11087.  7410. 12186.  2387. 12111.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  3395.  1087. 11904.  7232.  8840. 10115.  4494. 11516.  7441.
   8535. 12106.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.   494.     0.     0.  6541.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  8744. 11105.  1570.  5842.   174. 11266.  2929. 10438.  2879.
      0. 10936.  6330.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0. 11956.  5222.     0.     0. 12106.  6481.     0.  7093. 13756.
  12152.     0.     0.     0.     0. 10173.     0.  5173. 13756.  9371.
      0.  9956.     0.     0.  9716.     0.     0.     0.     0.     0.]
 [    0.  3395.  1087. 11904.  7232.  8840. 10115.  4494. 11516.  7441.
   8535. 12106.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.   420.  5842.  3058. 11875.  2879.  1087. 11105.  4995.  8840.
      0. 11100. 11875.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  5419.   420.  2250.  1299.  2151. 12111.  6454.     0. 11501.
   8094.  5842.   942.  7503.  7410.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.   420.  5842.  3058. 11875.  2879.  1087. 11105.  4995.  8840.
      0. 11100. 11875.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  3395.  1087. 11904.  7232.  8840. 10115.  4494. 11516.  7441.
   8535. 12106.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  1287.   420.  4070. 11087.  7410. 12186.  2387. 12111.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  3395.  1087. 11904.  7232.  8840. 10115.  4494. 11516.  7441.
   8535. 12106.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0. 11501.  1592. 10603. 11102.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.   174.  5842.  2387. 10453. 11090.     0.  7531. 11956.   450.
    420. 11516.  6693.  2624.  9963. 11992.  9322. 11090. 12106.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  7544.     0.  1709.   420. 10936.  5222.  5842. 10407.  6937.
  11329.  2937.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  1520.  1295.     0.  8396.  9322. 12715.     0.  5172.  7232.
  11266.     0. 11266.  2757.  4416. 12020. 12111.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  7544.     0.  1709.   420. 10936.  5222.  5842. 10407.  6937.
  11329.  2937.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.  9191.  5952.     0.     0. 11516.  9413.  3081.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0. 11516.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  3395.  1087. 11904.  7232.  8840. 10115.  4494. 11516.  7441.
   8535. 12106.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  9371. 10412.  2356.  5412. 11502.     0.  1087.   228.     0.
   2937. 11480. 10412.  5412.   420.  9435.  2937.   228.  1057.  9435.
  12111.     0.     0.     0.     0.     0.     0.     0.     0.     0.]]"""

In [ ]:
inputBatch = """[0. 12184.   420.  4636.     0.  8840.     0.     0. 10499. 11508.
   7511.     0.  5468.  2879.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]"""

In [ ]:
inputBatch.replace('.',',').replace('\n', '')

In [ ]:
import re
result = np.array(re.split("\s+",inputBatch.replace('[','').replace(']', '').replace('.','').replace('\n', '')), dtype=int).reshape(24, 30)

In [ ]:
inputBatch = result

In [ ]:
result.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

PositiveOrNegativeLabel=np.ones(shape = (24, 1))
inputBatch = result.reshape(24, 30, 1)


model=Sequential()
# input_shape is (num_timesteps, num_features)
model.add(LSTM(100,input_shape=(30, 1)))
model.add(Dense(1,activation="relu"))


model.compile(loss='mean_absolute_error',optimizer='adam')
model.fit(inputBatch,PositiveOrNegativeLabel,batch_size=24,epochs=9,verbose=1)

In [ ]:
firstSentence = np.array([  174, 11501,   420,  4242, 12111,     
                          0,     0,     0,     0,     0,     0,     0,
                          0,     0,     0,     0,     0,     0,     0])

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Need 30 timesteps
firstSentence = pad_sequences([firstSentence], maxlen = 30)[0]
firstSentence = firstSentence.reshape((1, 30, 1))

predict=model.predict(firstSentence, batch_size=1, verbose=1, steps=None)

# make a prediction
ynew = model.predict_classes(firstSentence)
# show the inputs and predicted outputs
for i in range(len(predict)):
    print("X=%s, Predicted=%s" % (predict[i], ynew[i]))

In [ ]:
predict=model.predict(firstSentence, batch_size=1, verbose=1, steps=None)
# make a prediction
ynew = model.predict_classes(firstSentence)
# show the inputs and predicted outputs
for i in range(len(predict)):
    print("X=%s, Predicted=%s" % (predict[i], ynew[i]))